In [1]:
import warnings
warnings.filterwarnings(action="ignore")

### 모델 관련 모듈

In [2]:
from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model

### 모델 정확도 검증 모듈

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

iris 데이터로 연습하면 좋다.

# 7. 유형별로 실습하는 머신러닝 P.34

In [4]:
import pandas as pd
import numpy as np

featuresData = pd.read_csv('../exampleCode/dataset/feature_regression_example.csv')
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


In [5]:
featuresData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

In [6]:
# featuresData.info

## 1. 타입 통합 ( (1) String 데이터를 Int 데이터로... / (2) Y/N 특성 데이터를 Int 데이터로...)

데이터가 5건이 넘어간다 -> 라벨인코더  
데이터가 몇 개 없다 -> np.where(속도 빠름)  
딕셔너리로 보관하고싶다 -> map

In [7]:
# 타입 통합 (P.36)
# YEARWEEK, YEAR, WEEK를 int 타입으로 설정해보고,
# 홀리데이 및 프로모션 여부 컬럼에 대해서 Y -> 1, N -> 0로 숫자형태로 나누어,
# HO_YN 컬럼과 PRO_YN 컬럼을 추가로 만드세요.

#### 1. 타입 통합

In [8]:
featuresData = featuresData.astype({"YEARWEEK":int,
                                    "YEAR":int,
                                    "WEEK":int})

#### 2. 라벨 생성 - 방법 (1) 라벨 인코더 사용하기.

In [9]:
%%time
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
featuresData["HO_YN"] = le.fit_transform(featuresData.HOLIDAY)
featuresData["PRO_YN"] = le.fit_transform(featuresData.PROMOTION)
featuresData

CPU times: user 1.01 ms, sys: 99 µs, total: 1.11 ms
Wall time: 1.06 ms


,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,0,1
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,0,1
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,0,1


#### 라벨인코더 검증해보기

In [10]:
# 검증 로직 Step 1. 역변환
featuresData['HO_YN_IV'] = le.inverse_transform(featuresData['HO_YN'])
featuresData['PRO_YN_IV'] = le.inverse_transform(featuresData['PRO_YN'])
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN,HO_YN_IV,PRO_YN_IV
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1,Y,Y
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1,N,Y
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1,N,Y
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1,Y,Y
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1,Y,Y
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,0,1,N,Y
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,0,1,N,Y
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,0,1,N,Y


In [11]:
# 검증 로직 Step 2. 역변환 값이 기존 값과 다른 것이 있는지 확인하기. (100% 원복된다면 불일치 값이 없어야 정상이다.)
featuresData.loc[featuresData.HOLIDAY != featuresData.HO_YN_IV]

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN,HO_YN_IV,PRO_YN_IV


In [12]:
featuresData.loc[featuresData.PROMOTION != featuresData.PRO_YN_IV]

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN,HO_YN_IV,PRO_YN_IV


#### 컬럼 드랍하기. (라벨 생성 다른 방법 실습을 위해서)

In [13]:
featuresData.drop(columns=['HO_YN','PRO_YN'], inplace=True)
del (featuresData['HO_YN_IV'],featuresData['PRO_YN_IV'])

featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


#### 2. 라벨 생성 - 방법 (2) np.where 사용하기.

In [14]:
%%time
featuresData['HO_YN'] = np.where(featuresData.HOLIDAY == 'Y',1,0)
featuresData['PRO_YN'] = np.where(featuresData.PROMOTION == 'Y',1,0)
featuresData

CPU times: user 1.13 ms, sys: 108 µs, total: 1.24 ms
Wall time: 1.18 ms


,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,0,1
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,0,1
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,0,1


#### 컬럼 드랍하기. (라벨 생성 다른 방법 실습을 위해서)

In [15]:
featuresData.drop(columns=['HO_YN','PRO_YN'], inplace=True)

featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


#### 2. 라벨 생성 - 방법 (3) map (딕셔너리)

In [16]:
%%time
binarymap = {"N":0, "Y":1}

featuresData["HO_YN"] = featuresData.HOLIDAY.map(binarymap)
featuresData["PRO_YN"] = featuresData.PROMOTION.map(binarymap)
featuresData

CPU times: user 1.8 ms, sys: 102 µs, total: 1.9 ms
Wall time: 1.82 ms


,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,0,1
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,0,1
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,0,1


#### 컬럼 드랍하기. (라벨 생성 다른 방법 실습을 위해서)

In [17]:
del (featuresData["HO_YN"],featuresData["PRO_YN"])
     
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


#### 2. 라벨 생성 - 방법 (4) 라벨 인코더로 더미 생성하기. (파생 변수 생성)

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [19]:
dummyData = pd.get_dummies(featuresData.HOLIDAY, prefix='HO')
dummyData

,HO_N,HO_Y
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
100,0,1
101,1,0
102,1,0
103,1,0


In [20]:
resultSet = pd.concat([featuresData, dummyData], axis=1)
resultSet

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_N,HO_Y
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,0,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,0,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,0,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,1,0
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,1,0
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,1,0


### 더미를 생성하는 방법은 한 컬럼에 Y, N의 정보를 모두 담을 수 없고, N인 컬럼, Y인 컬럼을 각각 만들어 True인 경우 체크를 하기 때문에 컬럼수가 다른 방법에 비해 2배가 된다.

## 2. Label(어떤 것을 예측할건지), Features(예측을 위해 어떤 것을 학습할건지) 선정하기.

In [21]:
featuresData["HO_YN"] = le.fit_transform(featuresData.HOLIDAY)
featuresData["PRO_YN"] = le.fit_transform(featuresData.PROMOTION)

#### Label 선정하기

In [22]:
labels = ['QTY']
labels

['QTY']

#### Features 선정하기

In [23]:
# 상관관계 확인하고 feature 찾기. (예측하려는 값이 QTY인데, 이것을 예측하는 모델을 만들기 위해 어떤 데이터를 학습 시켜야 좋을지? 즉, 연관성 높은 데이터가 어떤 것인지?를 찾는 과정이다.)
# QTY : 결과값 -> Label, 학습시키려는 연관성 높은 데이터 -> feature. 즉, f(x,y) = ax^2y + bxy^2 이란 공식에서 f(x,y)가 Label이고, x, y가 feature다.
corrDf = featuresData.corr()
corrDf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,HO_YN,PRO_YN
YEARWEEK,1.000000,0.956598,0.275593,0.112267,-0.071586,0.404889,0.049867,0.205916
YEAR,0.956598,1.000000,-0.016493,0.028931,0.028593,0.321193,-0.031106,0.195931
WEEK,0.275593,-0.016493,1.000000,0.289766,-0.339943,0.329705,0.273710,0.060206
QTY,0.112267,0.028931,0.289766,1.000000,-0.537230,0.700195,0.505932,0.612451
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619,-0.974601,-0.374072
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000,0.487062,0.898554
HO_YN,0.049867,-0.031106,0.273710,0.505932,-0.974601,0.487062,1.000000,0.365148
PRO_YN,0.205916,0.195931,0.060206,0.612451,-0.374072,0.898554,0.365148,1.000000


In [24]:
# 여기서 내가 원하는 상관관계 이상을 뽑아내보자.

In [25]:
# 상관관계 계수 비율 설정하기
corrStd = 0.5

In [26]:
# 상관관계가 0.5 < 상관관계 < 1 인 rows만 뽑아낸다.
targetIndex = corrDf.loc[ (abs(corrDf['QTY']) > corrStd) & (abs(corrDf.QTY) != 1),]
targetIndex

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,HO_YN,PRO_YN
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619,-0.974601,-0.374072
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000,0.487062,0.898554
HO_YN,0.049867,-0.031106,0.273710,0.505932,-0.974601,0.487062,1.000000,0.365148
PRO_YN,0.205916,0.195931,0.060206,0.612451,-0.374072,0.898554,0.365148,1.000000


In [20]:
features = list(targetIndex.index)
features

['HCLUS', 'PRO_PERCENT', 'HO_YN', 'PRO_YN']

## 3. 데이터 분리하기. (7:3 또는 8:2 맘에 드는거로...)

In [21]:
featuresData.YEARWEEK.describe

<bound method NDFrame.describe of 0      201501
1      201502
2      201503
3      201504
4      201505
        ...  
100    201648
101    201649
102    201650
103    201651
104    201652
Name: YEARWEEK, Length: 105, dtype: int64>

In [56]:
# 비율 정의
ratioStd = 0.8

In [57]:
# 위에서 정한 비율의 인덱스 구하기 (이후 이 갯수를 기준으로 분리가 잘 되었는지 검증할 때 사용)
indexStd = int(len(featuresData)*ratioStd)
indexStd

84

In [58]:
yearweekStd = featuresData.loc[indexStd,].YEARWEEK
yearweekStd

201632

In [59]:
trainingDataFeatures = featuresData.loc[featuresData.YEARWEEK < yearweekStd][features]
trainingDataLabel = featuresData.loc[featuresData.YEARWEEK < yearweekStd][labels]

In [60]:
testDataFeatures = featuresData.loc[featuresData.YEARWEEK >= yearweekStd][features]
testDataLabel = featuresData.loc[featuresData.YEARWEEK >= yearweekStd][labels]

In [61]:
# 위에서 분리한 비율 갯수가 맞는지 검증하기.
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)

(84, 4)
(84, 1)
(21, 4)
(21, 1)


## 4. 머신 러닝 (앞의 8의 데이터를 이용해 머신을 학습)

In [62]:
# 모델 정의
model_method_dt = tree.DecisionTreeRegressor(random_state=42)        # Decision Tree
model_method_rf = ensemble.RandomForestRegressor(random_state=42)    # Random Forest (Ensemble Model)

In [80]:
# 머신 러닝 (모델을 trainingData로 학습. Features -> Label)
model_dt = model_method_dt.fit(X=trainingDataFeatures,
                               y=trainingDataLabel)
model_rf = model_method_rf.fit(X=trainingDataFeatures,
                               y=trainingDataLabel)

## 5. 예측(테스트) (뒤의 2의 데이터를 이용해 머신을 테스트)

In [92]:
# 예측(시험) (위에서 학습시킨 머신 러닝 모델을 정답지가 있는 테스트 데이터에 적용해 검증한다)
# 모델 -> testDataFeatures(시험지)에 적용해 나온 predictValue(모델이 시험지 푼 결과) 결과를 실제 답지 testDataLabel(답지)와 비교해 정확도를 확인한다.
predictValueDt = model_dt.predict(testDataFeatures)   # 모델이 시험지 푼 결과
predictValueRf = model_rf.predict(testDataFeatures)

In [87]:
predictValueDt

array([1578.33333333,  350.71428571, 1434.6       , 1434.6       ,
       1434.6       , 1434.6       , 1434.6       , 1578.33333333,
       1578.33333333, 2620.42857143, 1578.33333333,  350.71428571,
       2193.8       , 2193.8       , 2193.8       , 2193.8       ,
       2193.8       , 1434.6       , 1434.6       , 1434.6       ,
       1434.6       ])

In [88]:
predictValueRf

array([1614.69339683,  347.45834533, 1435.3395    , 1435.3395    ,
       1435.3395    , 1410.84004365, 1410.84004365, 1614.69339683,
       1614.69339683, 2499.51268737, 1614.69339683,  347.45834533,
       2197.89016434, 2197.89016434, 2197.89016434, 2197.89016434,
       2197.89016434, 1640.17882215, 1640.17882215, 1640.17882215,
       1799.77588564])

In [93]:
# 모델이 시험지 풀어 제출한 갯수 확인하기.
len(predictValueDt)    # 8:2로 분리하면서 총 105개의 데이터가 84:21로 분리되었다. 6:4로 분리하면 총 105개의 데이터가 63:42로 분리되어 predictValue는 42가 나온다.

21

In [90]:
len(predictValueRf)

21

## 6. 데이터 정리. (답지(testDataLabel)과 머신러닝이 풀어서 제출한 결과(predictValue)를 비교해 정답률을 확인하기 위해 데이터를 정제)
(1) 길이가 동일하며  
(2) 인덱스가 리셋된  
(3) 데이터프레임으로 정제

In [91]:
# 예측 결과를 데이터프레임으로 변환
predictDf = pd.DataFrame(predictValueDt, columns=['DT_PREDICT'])
predictRf = pd.DataFrame(predictValueRf, columns=['RF_PREDICT'])

In [95]:
predictDf.head()

,DT_PREDICT
0,1578.333333
1,350.714286
2,1434.600000
3,1434.600000
4,1434.600000


In [96]:
predictRf.head()

,RF_PREDICT
0,1614.693397
1,347.458345
2,1435.339500
3,1435.339500
4,1435.339500


#### predictValue는 길이가 동일(위에서 확인했음), 인덱스가 리셋, 데이터프레임으로 정제 완료.
#### 이제 testDataLabel을 정제하자.

In [99]:
testDataLabel.head()

,QTY
84,2100
85,43
86,1700
87,1514
88,1501


In [100]:
# 인덱스 리셋
testDataLabel.reset_index(drop=True, inplace=True)

## 7. 결과 검증(정답률 확인)

Predict와 TestDataLebel를 병합한 후 비교한다

In [49]:
finalResultDf = pd.concat([testDataLabel, predictDf], axis=1)    # 인덱스가 달라 그냥은 잘 안 붙는다. testData를 reset_index 해야한다.
finalResultRf = pd.concat([testDataLabel, predictRf], axis=1)

In [49]:
finalResultDf

,QTY,DT_PREDICT
0,1700,1434.600000
1,1514,1434.600000
2,1501,1434.600000
3,1491,1434.600000
4,806,1434.600000
5,2111,1708.750000
6,2400,1708.750000
7,2010,2620.428571
8,1900,1708.750000
9,141,336.727273


In [ ]:
# 실제값 예측값을 파라미터로 던져주면 정확도 지표를 뽑아준다.

In [ ]:
mean_squared_error(finalResult.QTY, finalResult.DT_PREDICT)
mean_absolute_error(finalResult.QTY, finalResult.DT_PREDICT)

In [42]:
# 미래에 대한 특성 정보
HCLUS = 4
PRO_PERCENT = 0.3
HO_YN=1
PRO_YN=1

In [43]:
futureData = pd.DataFrame([[HCLUS, PRO_PERCENT, HO_YN, PRO_YN]],
                         columns = testDataFeatures.columns)

NameError: name 'HO_YN' is not defined

In [44]:
model.predict(featureData)

NameError: name 'model' is not defined

In [45]:
testDataFeatures

NameError: name 'testDataFeatures' is not defined

In [35]:
list(testDataLabel.index)

[86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106]

In [36]:
featuresDataLabel2 = featuresData.loc[list(testDataLabel.index)]

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
86,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34,1700,Y,1,Y,0.308584,1,1
87,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35,1514,Y,1,Y,0.308584,1,1
88,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201636,2016,36,1501,Y,1,Y,0.308584,1,1
89,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201637,2016,37,1491,N,4,Y,0.308584,0,1
90,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201638,2016,38,806,N,4,Y,0.308584,0,1
91,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201639,2016,39,2111,N,4,Y,0.280258,0,1
92,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201640,2016,40,2400,N,4,Y,0.280258,0,1
93,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201641,2016,41,2010,Y,2,Y,0.280258,1,1
94,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201642,2016,42,1900,N,4,Y,0.280258,0,1
95,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201643,2016,43,141,N,4,N,0.000000,0,0


In [ ]:
# 원래 concat을 해야하는데 이렇게 하면 더 쉽다.
featuresDataLabel2['DT_PREDICT'] = predictValue

In [ ]:
featuresDataLabel2

In [ ]:
featuresDataLabel2.to_csv('./dt_result.csv')

In [ ]:
엑셀 accurance 정확도 빼는거 앞뒤 바꾸면 과다예측, 과소예측... 예측 방향이 뒤바뀐다...

In [1]:
# sklearn의 트리모델 및 graphviz 라이브러리 선언
from sklearn.tree import export_graphviz
import graphviz

# graphviz로 tree.dot 출력
export_graphviz(decision_tree=model, 
                out_file="tree.dot",
                impurity=False)

# tree.dot 파일을 불러와서 화면에 시연
with open("tree.dot") as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

ModuleNotFoundError: No module named 'graphviz'

## 8. 결과 시각화 및 분석